<a href="https://colab.research.google.com/github/Afnan93/Afmnan/blob/main/DataExploremvtec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938113-1629952094/mvtec_anomaly_detection.tar.xz" -O mvtec_anomaly_detection.tar.xz


--2025-06-26 18:26:11--  https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938113-1629952094/mvtec_anomaly_detection.tar.xz
Resolving www.mydrive.ch (www.mydrive.ch)... 91.214.169.64
Connecting to www.mydrive.ch (www.mydrive.ch)|91.214.169.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5264982680 (4.9G) [application/x-xz]
Saving to: ‘mvtec_anomaly_detection.tar.xz’

mvtec_anomaly_detec 100%[===================>]   4.90G  18.1MB/s    in 4m 46s  

2025-06-26 18:30:58 (17.6 MB/s) - ‘mvtec_anomaly_detection.tar.xz’ saved [5264982680/5264982680]



In [3]:
import tarfile
with tarfile.open("mvtec_anomaly_detection.tar.xz", "r:xz") as tar:
    tar.extractall(path="mvtec_data")

print("Extraction complete.")


Extraction complete.


In [4]:
import os
from collections import defaultdict
root_dir = "/content/mvtec_data"

summary = {}

for category in os.listdir(root_dir):
    category_path = os.path.join(root_dir, category)
    if not os.path.isdir(category_path):
        continue

    train_good_path = os.path.join(category_path, "train", "good")
    test_path = os.path.join(category_path, "test")

    if not os.path.exists(train_good_path) or not os.path.exists(test_path):
        continue

    data = {
        "train_good": len(os.listdir(train_good_path)),
        "test_good": 0,
        "defect_types": {},
    }

    for test_type in os.listdir(test_path):
        test_type_path = os.path.join(test_path, test_type)
        if not os.path.isdir(test_type_path):
            continue

        num_images = len(os.listdir(test_type_path))
        if test_type == "good":
            data["test_good"] = num_images
        else:
            data["defect_types"][test_type] = num_images

    summary[category] = data

for category, data in summary.items():
    print(f"\nCategory: {category}")
    print(f"  Train images (good): {data['train_good']}")
    print(f"  Test images (good): {data['test_good']}")
    print(f"  Defect types:")
    for defect_type, count in data["defect_types"].items():
        print(f"    - {defect_type}: {count}")



Category: tile
  Train images (good): 230
  Test images (good): 33
  Defect types:
    - oil: 18
    - gray_stroke: 16
    - rough: 15
    - glue_strip: 18
    - crack: 17

Category: cable
  Train images (good): 224
  Test images (good): 58
  Defect types:
    - combined: 11
    - missing_wire: 10
    - cut_inner_insulation: 14
    - bent_wire: 13
    - missing_cable: 12
    - cable_swap: 12
    - poke_insulation: 10
    - cut_outer_insulation: 10

Category: zipper
  Train images (good): 240
  Test images (good): 32
  Defect types:
    - squeezed_teeth: 16
    - combined: 16
    - fabric_interior: 16
    - fabric_border: 17
    - split_teeth: 18
    - rough: 17
    - broken_teeth: 19

Category: metal_nut
  Train images (good): 220
  Test images (good): 22
  Defect types:
    - scratch: 23
    - flip: 23
    - bent: 25
    - color: 22

Category: grid
  Train images (good): 264
  Test images (good): 21
  Defect types:
    - glue: 11
    - thread: 11
    - broken: 12
    - bent: 12
    -

In [86]:
import os

dataset_path = "/content/mvtec_data"

summary = []

for category in sorted(os.listdir(dataset_path)):
    category_path = os.path.join(dataset_path, category)
    if not os.path.isdir(category_path):
        continue

    # Train Good
    train_good_dir = os.path.join(category_path, "train", "good")
    train_good_count = len(os.listdir(train_good_dir)) if os.path.exists(train_good_dir) else 0

    # Test Good
    test_good_dir = os.path.join(category_path, "test", "good")
    test_good_count = len(os.listdir(test_good_dir)) if os.path.exists(test_good_dir) else 0

    # Test Defective
    test_dir = os.path.join(category_path, "test")
    defect_count = 0
    for defect_type in os.listdir(test_dir):
        if defect_type == "good":
            continue
        defect_folder = os.path.join(test_dir, defect_type)
        defect_count += len(os.listdir(defect_folder))

    # Ground truth masks
    gt_dir = os.path.join(category_path, "ground_truth")
    gt_count = 0
    if os.path.exists(gt_dir):
        for subfolder in os.listdir(gt_dir):
            gt_folder = os.path.join(gt_dir, subfolder)
            gt_count += len(os.listdir(gt_folder))

    summary.append({
        "Category": category,
        "Train Good": train_good_count,
        "Test Good": test_good_count,
        "Test Defect": defect_count,
        "Ground Truth Masks": gt_count
    })

# Display
import pandas as pd
df = pd.DataFrame(summary)
df["Total Images"] = df["Train Good"] + df["Test Good"] + df["Test Defect"]
df = df.sort_values(by="Category").reset_index(drop=True)
df


,Category,Train Good,Test Good,Test Defect,Ground Truth Masks,Total Images
0,bottle,209,20,63,63,292
1,cable,224,58,92,92,374
2,capsule,219,23,109,109,351
3,carpet,280,28,89,89,397
4,grid,264,21,57,57,342
5,hazelnut,391,40,70,70,501
6,leather,245,32,92,92,369
7,metal_nut,220,22,93,93,335
8,pill,267,26,141,141,434
9,screw,320,41,119,119,480


In [103]:
import os
from PIL import Image

# Change this to your preffered object
category = "screw"
base_path = f"/content/mvtec_data/pill"
image_dirs = [
    os.path.join(base_path, "train", "good"),
    os.path.join(base_path, "test", "good"),
]


test_dir = os.path.join(base_path, "test")
if os.path.exists(test_dir):
    for defect_type in os.listdir(test_dir):
        if defect_type != "good":
            image_dirs.append(os.path.join(test_dir, defect_type))

#  ground truth masks
gt_dir = os.path.join(base_path, "ground_truth")
if os.path.exists(gt_dir):
    for defect_type in os.listdir(gt_dir):
        image_dirs.append(os.path.join(gt_dir, defect_type))


print("📋 Sample image metadata:\n Pill")
for folder in image_dirs:
    if not os.path.exists(folder):
        continue
    for fname in os.listdir(folder)[:3]:  # Check only 3 files per folder
        fpath = os.path.join(folder, fname)
        with Image.open(fpath) as img:
            print(f"📁 {folder.split('/')[-2:]} | 🖼 {fname} | "
                  f"Format: {img.format}, Size: {img.size}, Mode: {img.mode}")


📋 Sample image metadata:
 Pill
📁 ['train', 'good'] | 🖼 091.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['train', 'good'] | 🖼 190.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['train', 'good'] | 🖼 258.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['test', 'good'] | 🖼 004.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['test', 'good'] | 🖼 015.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['test', 'good'] | 🖼 003.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['test', 'combined'] | 🖼 004.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['test', 'combined'] | 🖼 015.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['test', 'combined'] | 🖼 003.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['test', 'scratch'] | 🖼 004.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['test', 'scratch'] | 🖼 015.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['test', 'scratch'] | 🖼 003.png | Format: PNG, Size: (800, 800), Mode: RGB
📁 ['test', 'pill_type'] | 🖼 004.png | Format: PNG, Size: 

# **Observation:**
1.Dataset does not include a CSV, JSON, or annotation file listing labels.
2.It includes 15 objects categories with its folder structure.
3.Train set are good images without any defects.
4.Test set are good and defect images.
5.Eevery defective image have binary masks in its corresponding ground truth to show the defective parts at pixel level.
6.This dataset designed for unsupervised anomly detection task.